In [40]:
import numpy as np
import gym
import gym_draughts
import pygame
from draught_surf import DraughtVisualiser, find_square

In [43]:
class PvPEngine:
    def __init__(self):
        self.env = gym.make('draughts-v0')
        
    def run(self):
        pygame.init()
        d_surf = pygame.display.set_mode((800,800))
        vis = DraughtVisualiser(800,6)
        vis.draw_board()
        d_surf.blit(vis.my_surf, (0,0))
        pygame.display.set_caption("Test!")
        vis.draw_from_grid(self.env.board.board)
        pygame.display.update()
        MOVETHISPIECE=0
        
        while True: # main game loop
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                if event.type == pygame.MOUSEBUTTONUP:

                    pos = pygame.mouse.get_pos()
                    square = find_square( (0,0), pos, 800/6)
                    MOVETHISPIECE=square
                if event.type==pygame.KEYDOWN:
                    key_name=pygame.key.name(event.key)
                    if key_name=='w':
                        self.env.step([MOVETHISPIECE[0], MOVETHISPIECE[1], 0])
                    if key_name=='a':
                        self.env.step([MOVETHISPIECE[0], MOVETHISPIECE[1], 1])
                    if key_name=='s':
                        self.env.step([MOVETHISPIECE[0], MOVETHISPIECE[1], 2])
                    if key_name=='d':
                        self.env.step([MOVETHISPIECE[0], MOVETHISPIECE[1], 3])

            vis.draw_from_grid(self.env.board.board)
            colour=pygame.Color("chocolate1")
            d_surf.blit(vis.my_surf, (0,0))
            pygame.display.update()

In [44]:
game = PvPEngine()
game.run()

SQUARE: 1 3
SQUARE: 1 3
SQUARE: 1 3
SQUARE: 1 3
SQUARE: 1 3
SQUARE: 1 3
SQUARE: 4 2
SQUARE: 1 3
SQUARE: 5 3
SQUARE: 1 1
SQUARE: 3 3
SQUARE: 0 0
SQUARE: 4 4
SQUARE: 2 2
SQUARE: 5 5
SQUARE: 0 4
SQUARE: 4 2
SQUARE: 2 4
SQUARE: 2 4
SQUARE: 2 4
SQUARE: 5 1
SQUARE: 0 2
SQUARE: 3 1
SQUARE: 1 3
SQUARE: 2 0
SQUARE: 3 3
SQUARE: 3 3
SQUARE: 2 2
SQUARE: 0 2
SQUARE: 4 4
SQUARE: 1 5
SQUARE: 1 1
SQUARE: 4 0
SQUARE: 4 0
SQUARE: 1 5
SQUARE: 0 2
SQUARE: 2 4
SQUARE: 3 3
SQUARE: 1 3
SQUARE: 3 3
SQUARE: 2 2
SQUARE: 4 2
SQUARE: 3 1
SQUARE: 5 3
SQUARE: 5 3
SQUARE: 4 0
SQUARE: 5 1
SQUARE: 2 2
SQUARE: 4 2
SQUARE: 1 1
SQUARE: 3 1
SQUARE: 0 2
SQUARE: 4 2
SQUARE: 1 1
SQUARE: 3 1
SQUARE: 1 3


error: display Surface quit

In [22]:
board = gym.make('draughts-v0')

In [23]:
board.render()

[[ 1.  0.  1.  0.  1.  0.]
 [ 0.  1.  0.  1.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [-1.  0. -1.  0. -1.  0.]
 [ 0. -1.  0. -1.  0. -1.]]


In [26]:
board.step([4,0,1])

(array([[ 1.,  0.,  1.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  1.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  0., -1.,  0.],
        [ 0., -1.,  0., -1.,  0., -1.]]),
 0.3,
 False,
 False)

In [28]:
board.step([1,1,3])

(array([[ 1.,  0.,  1.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  1.],
        [ 1.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  0., -1.,  0.],
        [ 0., -1.,  0., -1.,  0., -1.]]),
 0.3,
 False,
 False)